<a href="https://colab.research.google.com/github/ghoshankur102/Judge_It_Well/blob/main/Week2/LSTM_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AN LSTM MODEL TO DO SENTIMENT ANALYSIS ON MOVIE REVIEW**

Importing libraries and preprocessing the dataset

In [ ]:
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import zipfile



from google.colab import files
uploaded = files.upload()
# Load the CSV file into a DataFrame named 'df'
df = pd.read_csv('IMDB Dataset.csv')

# View the first 5 rows to verify
print(df.head())







# Basic cleaning function (moved here for correct variable definition order)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<br\s*/?>", " ", text)     # remove HTML breaks
    text = re.sub(r"http\S+|www\.\S+", " ", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)    # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()       # normalize spaces
    return text

# Extract texts and labels from the DataFrame
texts = df["review"].astype(str)
labels = df["sentiment"].apply(lambda x: 1 if x == 'positive' else 0).astype(int)

texts_clean = [clean_text(t) for t in texts]

#building voacbulary
vocab_size = 20000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(texts_clean)

#text -> integers
sequences = tokenizer.texts_to_sequences(texts_clean)

# truncate sequences
max_len = 200
X = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")
y = np.array(labels, dtype="int32")

Saving IMDB Dataset.csv to IMDB Dataset.csv
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
# Basic cleaning function

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<br\s*/?>", " ", text)     # remove HTML breaks
    text = re.sub(r"http\S+|www\.\S+", " ", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)    # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()       # normalize spaces
    return text
texts_clean = [clean_text(t) for t in texts]

In [ ]:
#building voacbulary
vocab_size = 20000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(texts_clean)

In [ ]:
import os
print(os.listdir('.'))

Spliting the dataset for training(about 70%, used for training)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:  ", X_val.shape, y_val.shape)
print("Test shape: ", X_test.shape, y_test.shape)

Training the dataset

In [ ]:
vocab_size = min(vocab_size, len(tokenizer.word_index) + 1)
embed_dim = 128
lstm_units = 128

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")  # binary sentiment
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train model
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs= 25,
    validation_data=(X_val, y_val)
)

Evalutating the model using test data

In [ ]:

#Evaluate on test set
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

#predict sentiment for test data
iter =  10
count = 0
for i in range(iter):
    x = X_test[i:i+1]
    true_label = y_test[i]

    prob = float(model.predict(x, verbose=0)[0][0])
    pred_label = 1 if prob >= 0.5 else 0
    if pred_label == true_label:
      count += 1

    print(f"True label: {true_label}  Pred prob (positive): {prob:.4f}")
    print("Predicted:", "POSITIVE" if pred_label == 1 else "NEGATIVE")
    print("")

print(f"Accuracy = {(100*count/iter) :.4f} %")
